#Подготовка датасета


Импортируем необходимые библиотеки

In [ ]:
import os
import random
import shutil
import time

from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

## Преобразование из png 278 на 278 в jpg 28 на 28

In [ ]:
# Этот код преобразует все изображения в папке из png 278 на 278 в jpg 28 на 28

# преобразуем изображение в jpg 28 на 28
def convert_img(path):
    png_image = Image.open(path)

    # Создаем новое пустое изображение с белым фоном такого же размера
    white_background = Image.new('RGB', png_image.size, (255, 255, 255))

    # Копируем исходное изображение на новое изображение с белым фоном
    white_background.paste(png_image, (0, 0), png_image)

    # делаем изображение 28 на 28
    new_image = white_background.resize((28, 28))

    # Преобразуем изображение в черно-белое
    new_image = new_image.convert('L')

    # Сохраняем новое изображение в формате JPEG
    new_image.save(path[:-4]+".jpg", 'JPEG')

    # удаляем старое
    os.remove(path)


# преобразуем все изображения в папке с датасетом
def normalize_images(input_dir):
    # Перебираем все файлы в директории
    print(input_dir)
    entries = os.listdir(input_dir)

    # Разделяем файлы и папки
    files = [entry for entry in entries if os.path.isfile(os.path.join(input_dir, entry))]
    folders = [entry for entry in entries if os.path.isdir(os.path.join(input_dir, entry))]

    for i in files:
        path = os.path.join(input_dir, i)
        convert_img(path)

    for i in folders:
        path = os.path.join(input_dir, i)
        normalize_images(path)


# Указываем директории датасета
input_dir = r'C:\Users\Sergey\PycharmProjects\FEFU_ML_2024\Cyrillic'

normalize_images(input_dir)


## Балансировка датасета с последующим разделением датасета на тренировочный, валидационный и тестовый наборы


In [ ]:
dataset = {}


def get_images_paths(input_dir, folder=""):
    # Перебираем все файлы в директории
    print(input_dir)
    entries = os.listdir(input_dir)

    # Разделяем файлы и папки
    files = [entry for entry in entries if os.path.isfile(os.path.join(input_dir, entry))]
    folders = [entry for entry in entries if os.path.isdir(os.path.join(input_dir, entry))]

    for i in files:
        path = os.path.join(input_dir, i)
        dataset.setdefault(folder, [])
        dataset[folder].append(path)

    for i in folders:
        path = os.path.join(input_dir, i)
        get_images_paths(path, i)


# Указываем директории с исходными и выходными изображениями
input_dir = r'C:\Users\Sergey\PycharmProjects\FEFU_ML_2024\Cyrillic'
output_dir = r'C:\Users\Sergey\PycharmProjects\FEFU_ML_2024'

output_dir_train_validation = os.path.join(output_dir, 'Cyrillic_Train_Validation')
output_dir_test = os.path.join(output_dir, 'Cyrillic_Test')

get_images_paths(input_dir)

for k in dataset.keys():
    print(k, len(dataset[k]))

min_count = [len(i) for i in dataset.values()]
min_count = min(min_count)
print("min_count:", min_count)


for key in dataset.keys():
    print(key)
    value = dataset[key]
    random.shuffle(value)

    train_validation_dataset_path = os.path.join(output_dir_train_validation, key)
    if not os.path.exists(train_validation_dataset_path):
        # Создаем директорию, если нет
        os.makedirs(train_validation_dataset_path)

    test_dataset_path = os.path.join(output_dir_test, key)
    if not os.path.exists(test_dataset_path):
        # Создаем директорию, если нет
        os.makedirs(test_dataset_path)

    for i in value[:30]:
        shutil.copy2(i, test_dataset_path)

    for j in value[30:min_count]:
        shutil.copy2(j, train_validation_dataset_path)


## Аугментацией датасета

Аугментация датасета играет важную роль в задачах машинного обучения, особенно в задачах, где данных недостаточно для обучения модели. Она позволяет увеличить размер датасета путем создания новых данных, основываясь на уже имеющихся.

In [ ]:
# увеличиваем датасет

def gen_variations(path_to_image, path_to_save_dir, number_of_variations=20):
    datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=False,
            fill_mode='nearest')

    img = load_img(path_to_image)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=path_to_save_dir,
                              save_prefix=f'gen_{str(time.time())}',
                              save_format='jpg'):
        i += 1
        if i > number_of_variations:
            break


files_and_folders = []
def get_images_paths(input_dir):
    # Перебираем все файлы в директории
    print(input_dir)
    entries = os.listdir(input_dir)

    # Разделяем файлы и папки
    files = [entry for entry in entries if os.path.isfile(os.path.join(input_dir, entry))]
    folders = [entry for entry in entries if os.path.isdir(os.path.join(input_dir, entry))]

    for i in files:
        path = os.path.join(input_dir, i)
        files_and_folders.append(
            (path, input_dir)
        )

    for i in folders:
        path = os.path.join(input_dir, i)
        get_images_paths(path)


input_dir = r'C:\Users\Sergey\PycharmProjects\FEFU_ML_2024\Cyrillic'
get_images_paths(input_dir)


for i in range(len(files_and_folders)):
    print(f"{i+1}/{len(files_and_folders)}")
    gen_variations(files_and_folders[i][0], files_and_folders[i][1])
